In [1]:
import pandas as pd
import numpy as np

In [9]:
x = pd.read_csv('x_train.csv', parse_dates=['date_recorded'])
y = pd.read_csv('y_train.csv')
full = pd.concat([x,y], axis=1)

### info about data:

In [11]:
len(x)

59400

In [12]:
x.head().T

,0,1,2,3,4
id,69572,8776,34310,67743,19728
amount_tsh,6000,0,25,0,0
date_recorded,2011-03-14 00:00:00,2013-03-06 00:00:00,2013-02-25 00:00:00,2013-01-28 00:00:00,2011-07-13 00:00:00
funder,Roman,Grumeti,Lottery Club,Unicef,Action In A
gps_height,1390,1399,686,263,0
installer,Roman,GRUMETI,World vision,UNICEF,Artisan
longitude,34.9381,34.6988,37.4607,38.4862,31.1308
latitude,-9.85632,-2.14747,-3.82133,-11.1553,-1.82536
wpt_name,none,Zahanati,Kwa Mahundi,Zahanati Ya Nanyumbu,Shuleni
num_private,0,0,0,0,0


In [13]:
x.nunique()

id                       59400
amount_tsh                  98
date_recorded              356
funder                    1897
gps_height                2428
installer                 2145
longitude                57516
latitude                 57517
wpt_name                 37400
num_private                 65
basin                        9
subvillage               19287
region                      21
region_code                 27
district_code               20
lga                        125
ward                      2092
population                1049
public_meeting               2
recorded_by                  1
scheme_management           12
scheme_name               2696
permit                       2
construction_year           55
extraction_type             18
extraction_type_group       13
extraction_type_class        7
management                  12
management_group             5
payment                      7
payment_type                 7
water_quality                8
quality_

In [21]:
import plotly.graph_objects as go
from sklearn import preprocessing
from sklearn.utils import resample

In [18]:
le = preprocessing.LabelEncoder()
labels = le.fit_transform(full['status_group'])

In [83]:
idx = resample(range(len(full)), n_samples=1000, replace=False, random_state=12)
sample_x = full.iloc[idx]
sample_y = labels[idx]

In [84]:
# 0 is functional, 1 needs repair, 2 non-functional

In [85]:
t_ = np.unique(labels, return_counts=True)[1]
t_/len(labels)

array([0.54308081, 0.07267677, 0.38424242])

In [86]:
t = np.unique(sample_y, return_counts=True)[1]
t/1000

array([0.545, 0.067, 0.388])

In [87]:
fig = go.Figure(data=go.Scattergeo(
        lon = sample_x['longitude'],
        lat = sample_x['latitude'],
        mode = 'markers',
        text = sample_x['status_group'],
        marker_color = sample_y,
        ))

In [88]:
fig.update_layout(
        geo_scope='africa',
    )
fig.show()